In [1]:
from tqdm.auto import tqdm
import pandas as pd
from os import listdir, remove
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import ufal.udpipe
from ufal.udpipe import Model, Pipeline, ProcessingError, OutputFormat, Sentence

In [2]:
# Путь к каталогам с входными и выходными файлами.
# Обрабатываются ВСЕ txt-файлы из входного катклога.
# Все результаты пишутся в выходной каталог.
input_dir = "data/Corpora to work"
output_dir = "data/output"

In [3]:
# Собираем список txt-файлов из входного каталога.
files = listdir(input_dir)
files = list(filter(lambda x: x.endswith('.txt'), files))
files = [input_dir + '/' + file for file in files]

In [4]:
# Считает частоты лемм во входном предложении.
def calc_freqs(sent_collu:str, freqs:dict):
    words = sent_collu.split('\n')
    for word in words:
        parts = word.split('\t')
        if len(parts) > 3:
            freqs[parts[2]] += 1

# Размечает предложения из входного файла при помощи подели UDPipe.
# Пишет результат в разметке CONLLU в выходной файл.
# Если передан словарь (defaultdict), считает частоты лемм.
def tag_file(in_filename:str, out_filename:str, model, freqs: dict=None):
    conlluOutput = OutputFormat.newOutputFormat("conllu")
    sentence = Sentence()
    error = ProcessingError();
    tokenizer = model.newTokenizer(model.DEFAULT)
    
    with open(in_filename, "r") as in_file, open(out_filename, "a") as out_file:
        print(in_filename)
        for line in in_file:
            if line.strip() == "":
                continue
            tokenizer.setText(line)
            while tokenizer.nextSentence(sentence, error):
                model.tag(sentence, Model.DEFAULT)
#                 model.parse(sentence, model.DEFAULT)
                res_sent = conlluOutput.writeSentence(sentence)
                out_file.write(res_sent)
                if freqs != None:
                    calc_freqs(res_sent, freqs)


In [5]:
# Открываем модель.
modelfile = "/home/edward/projects/Alien_bases/Universal Dependencies/russian-syntagrus-ud-2.5-191206.udpipe"
model = Model.load(modelfile)

In [6]:
# Удаляем старый файл с разметкой.
remove(output_dir+'/tagged_result.txt')

freqs = defaultdict(int)
# Для всех файлов в каталоге проводим разметку.
for file in files:
    try:
        tag_file(file, output_dir+'/tagged_result.txt', model, freqs)
    except Exception:
        print('Problems with', file)

data/Corpora to work/1. é. ƒ. à½∞¼ÑÑó. é »á¼∩Γ¿ πτÑ¡¿¬«ó.txt
data/Corpora to work/10. ÉÑßπαß¡δ⌐ »«ñσ«ñ ó ¿ºπτÑ¡¿¿ α«½¿ ¡áÑ¼¡δσ αáí«Γ¡¿¬«ó ó ß«µ¿á½∞¡«¼ »áαΓ¡ÑαßΓóÑ.txt
data/Corpora to work/11. æΓÑαÑ«Γ¿»δ ó¡ÑΦ¡«ßΓ¿ ó« ß«óαÑ¼Ñ¡¡«⌐ ñÑΓß¬«⌐ α«ßß¿⌐ß¬«⌐ αÑá¡¿¼áµ¿¿.txt
data/Corpora to work/12. Æαá¡ß¡áµ¿«¡á½∞¡δ⌐ »«ñσ«ñ ó ß«óαÑ¼Ñ¡¡δσ ¼¿úαáµ¿«¡¡δσ ¿ßß½Ññ«óá¡¿∩σ.txt
data/Corpora to work/2. é. ƒ. à½∞¼ÑÑó - «ß¡«óáΓÑ½∞ ¬«¡µÑ»µ¿¿ »α¿¬½áñ¡«⌐ ß«µ¿«½«ú¿¿ ó æÅíâô.txt
data/Corpora to work/3. æ«µ¿«½«ú¿∩ ß«íßΓóÑ¡¡«ßΓ¿ «Γ ¿ñÑ⌐ è. îáα¬ßá ¬ »ÑΓÑαíπαúß¬«⌐ Φ¬«½Ñ φ¬«¡«¼¿τÑß¬«⌐ ß«µ¿«½«ú¿¿.txt
data/Corpora to work/4. ÄΓ αÑñá¬µ¿¿.txt
data/Corpora to work/5. Å«ó«α«Γ ¬ í¿«½«ú¿¿. ÅÑαß»Ñ¬Γ¿óδ αáºó¿Γ¿∩ ß«µ¿«½«ú¿τÑß¬«ú« º¡á¡¿∩.txt
data/Corpora to work/6. ü¿»«½∩α¡«Ñ αáßßΓα«⌐ßΓó«. ¥»¿ñÑ¼¿∩ ¿½¿ ß½ÑñßΓó¿Ñ ¼Ññ¿¬á½¿ºáµ¿¿ «íΘÑßΓóá.txt
data/Corpora to work/7. æΓ¿ú¼áΓ¿ºáµ¿∩ ½εñÑ⌐, ª¿óπΘ¿σ ß éêù, æÅêä ¿ »α«í½Ñ¼á αáß¬αδΓ¿∩ ßΓáΓπßá.txt
data/Corpora to work/8. î«ñÑ½∞ »α¿¡∩Γ¿∩ ΓÑαá»ÑóΓ¿τÑß¬¿σ αÑΦÑ¡¿⌐ ó ¬«ú¡¿Γ¿ó¡«⌐ ¼Ññ¿µ¿¡ß¬«⌐ á¡Γα«»«½«ú

data/Corpora to work/Glebkina_1195.txt
Problems with data/Corpora to work/Glebkina_1195.txt
data/Corpora to work/Glebkina_1196.txt
data/Corpora to work/Glebkina_1198.txt
data/Corpora to work/Glebkina_1200.txt
data/Corpora to work/Glebkina_1216.txt
data/Corpora to work/Glebkina_1217.txt
Problems with data/Corpora to work/Glebkina_1217.txt
data/Corpora to work/Glebkina_1218.txt
data/Corpora to work/Glebkina_1220.txt
Problems with data/Corpora to work/Glebkina_1220.txt
data/Corpora to work/Glebkina_1221.txt
data/Corpora to work/Glebkina_1222.txt
data/Corpora to work/Glebkina_1223.txt
data/Corpora to work/Glebkina_1224.txt
data/Corpora to work/Glebkina_1259.txt
data/Corpora to work/Glebkina_1260.txt
Problems with data/Corpora to work/Glebkina_1260.txt
data/Corpora to work/Glebkina_1261.txt
data/Corpora to work/Glebkina_1262.txt
data/Corpora to work/Glebkina_1263.txt
data/Corpora to work/Glebkina_1264.txt
data/Corpora to work/Glebkina_1265.txt
Problems with data/Corpora to work/Glebkina_126

In [7]:
# Сортируем частотность лемм по убыванию.
# !!! Сейчас леммы с одинаковой частотой не отсортированы по алфавиту!!!
freqs2 = [str(y[0])+";"+str(y[1]) for y in sorted(freqs.items(), key = lambda x: x[1], reverse=True)]
# Пишем результат в файл.
with open(output_dir+'/freq_dict.csv', 'w') as out_file:
    out_file.write('\n'.join(freqs2))